# GloVe: Global Vectors for Word Representation
Paper Summary - CS 269 [Seminar: Machine Learning in Natural Language Processing](https://uclanlp.github.io/CS269-17/overview)  
Ziye Xing

The semantic vector space models play the fundamental yet crucial roles in a variety of natural language applications, since the distributional representation embeds each word into a real-valued vector that somehow capture the underlying semantic or syntactic regularities.

There are two major model families which have been used widely in different applications and fields in NLP, one of them is statistical based word vector and another is prediction based generative vector model. The former is represented by Latent Semantic Analysis model which takes the advantage of the global statistic information, while these models often have a hard time dealing with the intrinsic relationship between words. The latter one, origined from neural network, represented by skip-gram model, predicts the probability of words based on the local context window. Our new model GloVe takes the advantage of both approaches. Thus, it is worthwhile to learn a little background knowledge about these models.

## Matrix Factorization Methods
One of the most common matrix factorization methods is Latent Semantic Analysis or Latent Semantic Indexing in the information retrieval context. LSA constructs the matrices based on “term-document” type, which is a desired structure for query searching. It performs SVD on the matrices and decompose them and only keeps the top rank value in singular matrix. In such way, we compress and reconstruct the matrices by keeping only the major components and capture statistical information about the corpus.
<img src="resources/svd.jpg" alt="SVD" style="width: 500px;"/>

The Hyperspace Analogue to Language, on the other hand, utilizes the “term-term” co-occurrence matrices. Any time two words are next to each other, the association between them should be stronger. The shortcoming of naïve HAL is also obvious: the co-occurrence with *the*, *and* convey very little information about their similarity. There are several techniques available to address this issue and improve the performance.

## Shallow Window-Based Methods
Another main approach for word embedding is based on neural network. The word vector is learned within local context windows. The idea of neural network is not new, but it is not until recent to achieve a successful implementation for word representation. The `word2vec` model, consists of two different approaches, skip-gram and continuous bag-of-words (CBOW) models, is proposed to achieve the learning process on a single-layer architecture based on the inner product between two word vectors.
<img src="https://i.stack.imgur.com/O2YeO.png" alt="w2v" style="width: 500px;"/>

As shown in the graph above, the CBOW model predicts a word given the context window words as input. While, on the other hand, the skip-gram predicts a word's context words by giving the center word itself. `word2vec` also constructs Huffman tree and Negative Sampling to reduce the training cost for this model.

## The GloVe Model
While neither of the methods mentioned above did take the full advantages of both statistical global information and the local sub-optimal context estimation, GloVe model, which stands for Global Vectors, tries a new approach to capture the statistics from global corpus directly.

This is the step-by-step process that shows how to construct this model, we will first establish some notation for our problem. We want to construct a word-word co-occurrence matrix $X$, where
* $X_{ij}$ tabulate the number of times word $j$ occurs in the context of word $i$.

For example, consider a corpus:
> I love NLP and NLP is fun

There are merely one sentence and 6 words in that corpus. If we consider a window size as 5, we will have the following windows:

Window No. | Center Word | Content
:--------: | :---------: | -------
0 | I | I love NLP
1 | love | I love NLP and
2 | NLP | I love NLP and NLP
3 | and | love NLP and NLP is
4 | NLP | NLP and NLP is fun
5 | is | and NLP is fun
6 | fun | NLP is fun

Take window No.1 for instance, the center word is *love*, the context words are *I*, *NLP*, *and*. There are less than 2 words left to the center word, so there are only 3 context words in this case. We will then make an increment to $X_{love,I}$, $X_{love,NLP}$, and $X_{love,and}$.

Then let's define a few more notations:
* $X_i = \sum_kX_{ik}$, the number of times any word appears in the context of word $i$. In the example above $X_{love}$ is 3.
* $P_{ij} = P(j|i) = \frac{X_{ij}}{X_i}$, the probability that word $j$ appears in the context of word $i$.
* $ratio_{ijk} = P(k|i)/P(k|j) = \frac{P_{ik}}{P_{jk}}$, the ratio of probability between word $i$ and $j$ regarding to the context of word $k$.

In the paper, the author crafts a example to illustrate how the co-occurrence could be useful. Suppose we are interested in the concept of thermodynamic phase, so let target words $i = ice$ and $j = steam$. With selected context words from a 6 billion token corpus, the co-occurrence probabilities are shown as following:
<img src="https://nlp.stanford.edu/projects/glove/images/table.png" alt="glove" style="width: 500px;"/>

An interesting observation is that for context words like *water* and *fashion*, they are either related to both target words or to either of them. These non-discriminative words cancel out the ratio which results as close to 1. However, for word like *solid* which correlates with ice well, the ratio becomes much greater than 1, while for word like *gas* correlates with steam, the ratio becomes very small value. To be more generalized, we have observed a regularity can summarized by table like this:

Value of $ratio_{ijk}$ | $j, k$ related | $j, k$ not related
:--------: | :---------: | :-----:
**$i, k$ related** | $\approx1$ | $\gg1$
**$i, k$ not related** | $\ll1$ | $\approx1$

This observation gives us a good hint and starting point to construct our word vector model. We would like to construct a function that capture or encode the ratio information well in the vector space,
$$F(w_i,w_j,\tilde{w}_k) = \frac{P_{ik}}{P_{jk}}$$

Here $\tilde{w}$ means a separate context word vector. However, there is an issue about this model. There are 3 word vectors involved in it, which means the computation intensity and complexity of learning this model are mostly impossible to be afforded. With that in mind, considering the properties of our model, we could have the following modifications:
1. Since the vector space are inherently linear and we are considering the similarity in that space, the most natural way is to take the difference on $wi$ and $w_j$, without loss of linearity. Its form will look like this: $F(w_i-w_j, \tilde{w}_k)$.
2. It is worthful to notice that the arguments of our function are vectors, while the right-hand side is a scalar. It would be reasonable to take the dot product of the arguments to avoid the complicated parameterization. Now we have function formulated as: $F((w_i-w_j)^T\tilde{w}_k)$.
3. Note that for word-word co-occurrence matrices, the distinction between a word and a context word is arbitrary and exchangeable. In order to restore this symmetry, we first require $F$ be a homomorphism between the groups $(\mathbb{R}, +)$ and $(\mathbb{R}_{\gt{0}},\times)$, i.e., we want $F((w_i-w_j)^T\tilde{w}_k)$ equals to $F(w_i^T\tilde{w}_k)/F(w_j^T\tilde{w}_k)$, which could be sovled by $F(w_i^T\tilde{w}_k) = P_{ik}$ and $F = exp$, or, $w_i^T\tilde{w}_k = log(P_{ik}) = log(X_{ik}) - log(X_i)$.
4. This equation is still not symmetric, i.e., the left-hand side has symmetry but not the right-hand side because of the $log(X_i)$ term. We can add bias terms to cancel it out. Finally, we will have,
$$w_i^T\tilde{w}_k + b_i + \tilde{b}_k = log(X_{ik})$$

This is a much more simplified eqation comparing to the original proposal. There is still a drawback if we weight every co-occurrence equally, even if rare one. Thus, in the cost function we need to add a weight term to address this issue. Now, we have the cost function for the model:
$$J = \sum_{i,j=1}^V{f(X_{ij})(w_i^T\tilde{w}_k + b_i + \tilde{b}_k - log(X_{ik}))^2}$$

From experiments, the author found it would be a decent choice for $f(x) = (x/xmax)^{0.75}$, if $x \lt xmax$, where $xmax$ is around 100. Otherwise, $f(x) = 1$.

## Experiment
The evaluation methods tested in this paper are conducted on the word analogy task of previous work, word similarity tasks, and on some common benchmarks for word vector.

### Word analogies
The word analogy task is to answer question in the form of "a is to b as c is to what?" It can be divided into two subset, semantic questions and syntactic questions. The semantic questions are analogies about object or noun, such as people or places, while syntactic questions are analogies about verb tenses or forms of adjectives.
<img src="resources/glove-word-analogy-results.png" width="300px"/>

The table above are the results on the word analogy task, the best score are underlined. This task is also the primary focus for this model proposed since the model is built following the vector difference structure. It is clear to note that the analogies in vector space are mostly parallel to each other.

| Comparisons | man -> woman             |  city -> zip | comparative -> superlative |
| :-: | :----------------------:|:-----------------------:|:-----------------------:|
| GloVe Geometry | <img src="http://nlp.stanford.edu/projects/glove/images/man_woman_small.jpg"></img>  | <img src="http://nlp.stanford.edu/projects/glove/images/city_zip_small.jpg"></img> | <img src="http://nlp.stanford.edu/projects/glove/images/comparative_superlative_small.jpg"></img> |

### Word similarity
The word similarity task measures the linguistic or semantic similarity of the corresponding words. In the benchmark, all vectors are 300-dimensional. This clustering task reveals some words rarely used by human but related to the target word. The following table shows the nearest neighbors of *frog*:

| nearest neighbors of <br/> *frog* | Litoria             |  Leptodactylidae | Rana | Eleutherodactylus |
| :-: | :-----------------------------: | :-----------------: | :--------------: | :-----------------: |
| Pictures | <img src="http://nlp.stanford.edu/projects/glove/images/litoria.jpg"/> | <img src="http://nlp.stanford.edu/projects/glove/images/leptodactylidae.jpg"/> | <img src="http://nlp.stanford.edu/projects/glove/images/rana.jpg"/> | <img src="http://nlp.stanford.edu/projects/glove/images/eleutherodactylus.jpg"/> |

### Vector Length and Context Size
The experiment also shows how the vector length and context size will affect the performance of the model. A context window that extends to both left and right side of a target word is called symmetric. The one only extends to previous (left) words is called asymmetric. While most of the relationships are clear and easy to understand, it is interesting to notice that the syntactic sub-task for small and asymmetric context windows has better performance.

<img src="resources/6-Figure2-1.png" width="600px"/>

### Comparison to `word2vec`
There is a debate about it is either GloVe or `word2vec` the better representation model, although in the paper and experiment environment, it claims Glove outperforms `word2vec`. There are many other experiments in practice that comparing the performances between `word2vec` and GloVe. It is generally true that GloVe can produce better accuracy, but `word2vec` is a lighter tool that consumes less memory to achieve similar result in a reasonable period of time.
<img src="resources/glove-vs-word2vec.png" width="600px"/>

## References
* Landauer, Thomas K., Peter W. Foltz, and Darrell Laham. "An introduction to latent semantic analysis."
* Mikolov, Tomas, et al. "Distributed representations of words and phrases and their compositionality." Advances in neural information processing systems. 2013.
* Pennington, Jeffrey, Richard Socher, and Christopher Manning. "[Glove: Global vectors for word representation.](https://nlp.stanford.edu/pubs/glove.pdf)" EMNLP. 2014.
* [NLP Stanford](https://nlp.stanford.edu/projects/glove/)
* [GloVe vs word2vec revisited.](http://dsnotes.com/post/glove-enwiki/)